In [8]:
# inladen benodigde packages
import pandas as pd
import math

In [6]:
# inladen dataset met kolom voor scores
# inladen data
dataset_path = "~/share/Milena/programma-discriminatie-en-racisme/01_datapreparatie/"
dataset = pd.read_csv(dataset_path + 'dataset_dummy_empty.csv', sep = ';') # voor dummy dataset importeer eerste paar instances

In [12]:
# aantal zinnen te labellen pp = zinnen * 3 / n_annotators
# gat tussen labellaars = aantal_te_labellen / 3

def split_annotated_data(data, n_annotators):
    datasets = []
    aantal_to_label = math.ceil(len(data) * 3 / n_annotators) # omhoog afronden
    distance = round(aantal_to_label / n_annotators)
    
    for n in range(n_annotators):
        start = n * distance
        eind = start + aantal_to_label
        dataset = data.iloc[start:eind]
        datasets.append(dataset)
    return datasets

split_data = split_annotated_data(data = dataset, n_annotators = 5)
print(len(split_data[0]))
print(len(dataset))

3800
6333


In [14]:
# output genereren: iedere dataset naar .csv
for i, dataset in enumerate(split_data):
    dataset.to_csv('te_labellen_' + str(i) + '.csv')